In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install rioxarray
import numpy as np
import random
import os
import rasterio
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

data = np.load("/content/drive/MyDrive/NEW_MODIS_Combined/new_data.npz")
X = data["X"]
y = data["y"]

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 58.8 MB/s eta 0:00:00


# ONLY ONE PERSON NEEDS TO RUN THE CODE

## Fire Data

In [ ]:
import ee
import rioxarray

ee.Authenticate()
ee.Initialize(project="ee-ashleys10125")

In [ ]:
# Define the image collection and region
roi = ee.Geometry.Rectangle([139.4462100998403, -39.517965336636436, 154.7391788498403,-30.46423194050125])

start_date = '2023-11-01'
end_date = '2024-01-01'

In [ ]:
fire_ic = ee.ImageCollection("MODIS/061/MOD14A1") \
    .filterDate(start_date, end_date) \
    .filterBounds(roi) \
    .select("FireMask")


## Temperature Data

In [ ]:
lst_ic = ee.ImageCollection("MODIS/061/MOD11A1") \
    .filterDate(start_date, end_date) \
    .filterBounds(roi) \
    .select("LST_Day_1km")

## Spectral Data

In [ ]:
ref_ic = ee.ImageCollection("MODIS/061/MOD09GA") \
    .filterDate(start_date, end_date) \
    .filterBounds(roi) \
    .select(['sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03'])  # Red, NIR, Blue

## Combine and Download

In [ ]:
def process_image(date_img):
    img = ee.Image(date_img)

    date = img.date()

    # Match corresponding images from other collections
    red_nir = ref_ic.filterDate(date, date.advance(1, 'day')).first()
    fire = fire_ic.filterDate(date, date.advance(1, 'day')).first()
    lst = lst_ic.filterDate(date, date.advance(1, 'day')).first()

    # Compute NDVI
    red = ee.Image(red_nir).select('sur_refl_b01').multiply(0.0001)
    nir = ee.Image(red_nir).select('sur_refl_b02').multiply(0.0001)
    blue = ee.Image(red_nir).select('sur_refl_b03').multiply(0.0001)
    ndvi = nir.subtract(red).divide(nir.add(red)).rename("NDVI")

    #EVI (Enhanced Vegetation Index)
    evi = nir.subtract(red).divide(nir.add(red.multiply(6)).subtract(blue.multiply(7.5)).add(1).multiply(2.5)).rename("EVI")



    # Combine all bands
    combined = ee.Image.cat([ndvi, fire, lst, evi]) \
        .clip(roi) \
        .set('system:time_start', date.millis())

    return combined

# Use MOD14A1 as reference dates
dates = fire_ic.toList(fire_ic.size())

# Loop over images
nimg = fire_ic.size().getInfo()

for i in range(nimg):
    img = ee.Image(dates.get(i))
    date_str = img.date().format('yyyy-MM-dd').getInfo()

    combined_img = process_image(img)

    task = ee.batch.Export.image.toDrive(
        image=combined_img.toFloat(),
        description=f"combined_{date_str}",
        folder='NEW_MODIS_Combined',
        fileNamePrefix=f"combined_{date_str}",
        region=roi,
        dimensions=(1024, 1024),
        maxPixels=1e10
    )
    task.start()

# Connect to Google Drive

In [ ]:
file_path = '/content/drive/MyDrive/NEW_MODIS_Combined/combined_2023-11-25.tif'
# image = rioxarray.open_rasterio(file_path)
# #image = rioxarray.open_rasterio('2024-01-01.tif')
# dem = image[0] #getting the first band

# dem.plot()

### NDVI:
+1 is health, 0 is rock or soil, -1 is water or snow

### Fire Mask:


1.   Not processed (obsolete; not used since Collection 1)
2.   Not processed (other reason)
3.   Non-fire water pixel
4.   Cloud (land or water)
5.   Non-fire land pixel
6.   Unknown (land or water)
7.   Fire (low confidence, land or water)
8.   Fire (nominal confidence, land or water)
9.   Fire (high confidence, land or water)

### LST:
Kelvin temperature, minimum value is 7500; maximum value is 65535.
Scale is 0.02 (meaning actual temperature is value x 0.02)


In [ ]:
np.nanmean(ndvi)

NameError: name 'ndvi' is not defined

In [ ]:
with rasterio.open(file_path) as src:
    print(f"Number of bands: {src.count}")
    print(f"CRS: {src.crs}")
    print(f"Resolution: {src.res}")

    # Read all bands into a NumPy array
    bands = src.read()

    # Read individual bands (1-based indexing)
    ndvi = src.read(1)  # Band 1: NDVI
    firemask = src.read(2)  # Band 2: FireMask
    lst = src.read(3)  # Band 3: Land Surface Temp

    fire_mask_cmap = mcolors.ListedColormap([
        'white',    # 1: Not processed
        'gray',     # 2: Not processed
        'blue',     # 3: Non-fire water
        'lightblue', # 4: Cloud
        'green',    # 5: Non-fire land
        'yellow',   # 6: Unknown
        'orange',   # 7: Fire (low confidence)
        'red',      # 8: Fire (nominal confidence)
        'darkred'   # 9: Fire (high confidence)
    ])
    bounds = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


# Plot the data
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# NDVI plot
cax1 = axes[0].imshow(ndvi, cmap='YlGn', vmin=-1, vmax=1)
axes[0].set_title("NDVI")
fig.colorbar(cax1, ax=axes[0], orientation='vertical', label="NDVI Value")

# Fire Mask plot
cax2 = axes[1].imshow(firemask, cmap=fire_mask_cmap, vmin=1, vmax=9)
axes[1].set_title("Fire Mask")
fig.colorbar(cax2, ax=axes[1], orientation='vertical', label="Fire Mask Value")

# LST plot
cax3 = axes[2].imshow(lst, cmap='coolwarm')
axes[2].set_title("LST")
fig.colorbar(cax3, ax=axes[2], orientation='vertical', label="Temperature (K)")

plt.tight_layout()
plt.show()

NameError: name 'rasterio' is not defined

In [ ]:
# Save Data
dataset_folder = "/content/drive/MyDrive/MODIS_Combined/"
fire_save_folder = "/content/drive/MyDrive/MODIS_Combined/patches/fire"
nofire_save_folder = "/content/drive/MyDrive/MODIS_Combined/patches/nofire"

os.makedirs(fire_save_folder, exist_ok=True)
os.makedirs(nofire_save_folder, exist_ok=True)

# Get list of .tif files
tif_files = [f for f in os.listdir(dataset_folder) if f.endswith('.tif')]

patch_size = 32

for tif_file in tif_files:
    file_path = os.path.join(dataset_folder, tif_file)

    with rasterio.open(file_path) as src:
        ndvi_mask = src.read(1)  # Band 1: NDVI
        fire_mask = src.read(2)  # Band 2: FireMask
        temp_mask = src.read(3)  # Band 3: LST

    height, width = fire_mask.shape
    num_rows = height // patch_size
    num_cols = width // patch_size

    patch_index = 0
    for i in range(num_rows):
        for j in range(num_cols):
            y_start, y_end = i * patch_size, (i + 1) * patch_size
            x_start, x_end = j * patch_size, (j + 1) * patch_size

            fire_patch = fire_mask[y_start:y_end, x_start:x_end]
            ndvi_patch = ndvi_mask[y_start:y_end, x_start:x_end]
            temp_patch = temp_mask[y_start:y_end, x_start:x_end]

            mostly_empty = np.sum((fire_patch < 4)) / fire_patch.size > 0.80  # Mostly water/cloud/unprocessed
            fire_present = np.any(np.isin(fire_patch, [7, 8, 9]))  # Fire classes

            if mostly_empty:
                continue

            if fire_present:
                np.save(os.path.join(fire_save_folder, f"Fire_{tif_file}_patch_{patch_index}.npy"), fire_patch)
                np.save(os.path.join(fire_save_folder, f"NDVI_{tif_file}_patch_{patch_index}.npy"), ndvi_patch)
                np.save(os.path.join(fire_save_folder, f"Temp_{tif_file}_patch_{patch_index}.npy"), temp_patch)
            else:
                np.save(os.path.join(nofire_save_folder, f"Fire_{tif_file}_patch_{patch_index}.npy"), fire_patch)
                np.save(os.path.join(nofire_save_folder, f"NDVI_{tif_file}_patch_{patch_index}.npy"), ndvi_patch)
                np.save(os.path.join(nofire_save_folder, f"Temp_{tif_file}_patch_{patch_index}.npy"), temp_patch)

            patch_index += 1

In [ ]:
# File paths
patch_no = 23
file_date = "2023-11-05.tif"
fire = "fire" # "fire" and "nofire"
fire_patch_file = f"/content/drive/MyDrive/MODIS_Combined/patches/{fire}/Fire_combined_{file_date}_patch_{patch_no}.npy"
ndvi_patch_file = f"/content/drive/MyDrive/MODIS_Combined/patches/{fire}/NDVI_combined_{file_date}_patch_{patch_no}.npy"
lst_patch_file = f"/content/drive/MyDrive/MODIS_Combined/patches/{fire}/Temp_combined_{file_date}_patch_{patch_no}.npy"

firemask = np.load(fire_patch_file)
ndvi = np.load(ndvi_patch_file)
lst = np.load(lst_patch_file)

# Scale LST
lst = lst * 0.02  # Convert to Kelvin

# FireMask colormap
fire_mask_cmap = mcolors.ListedColormap([
    'white',     # 1: Not processed
    'gray',      # 2: Not processed
    'blue',      # 3: Non-fire water
    'lightblue', # 4: Cloud
    'green',     # 5: Non-fire land
    'yellow',    # 6: Unknown
    'orange',    # 7: Fire (low confidence)
    'red',       # 8: Fire (nominal confidence)
    'darkred'    # 9: Fire (high confidence)
])
bounds = np.arange(1, 10)
norm = mcolors.BoundaryNorm(bounds, fire_mask_cmap.N)

# Plotting
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

# NDVI
cax1 = axes[0].imshow(ndvi, cmap='YlGn', vmin=-1, vmax=1)
axes[0].set_title("NDVI")
fig.colorbar(cax1, ax=axes[0], label="NDVI")

# Fire Mask
cax2 = axes[1].imshow(firemask, cmap=fire_mask_cmap, norm=norm)
axes[1].set_title("Fire Mask")
fig.colorbar(cax2, ax=axes[1], ticks=bounds, label="Fire Class")

# LST
cax3 = axes[2].imshow(lst, cmap='coolwarm')
axes[2].set_title("Land Surface Temp (K)")
fig.colorbar(cax3, ax=axes[2], label="Kelvin")

plt.tight_layout()
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/MODIS_Combined/patches/fire/Fire_combined_2023-11-25.tif_patch_23.npy'

# Training Data


In [ ]:
from scipy.ndimage import rotate

def normalize_patch(ndvi_patch, temp_patch, evi_patch):
    ndvi_patch = np.nan_to_num(ndvi_patch, nan=0)
    temp_patch = np.nan_to_num(temp_patch, nan=0)
    evi_patch = np.nan_to_num(evi_patch, nan=0)

    # Normalize each patch using min-max scaling (0 to 1)
    ndvi_min, ndvi_max = np.nanmin(ndvi_patch), np.nanmax(ndvi_patch)
    temp_min, temp_max = np.nanmin(temp_patch), np.nanmax(temp_patch)
    evi_min, evi_max = np.nanmin(evi_patch), np.nanmax(evi_patch)

    # Avoid division by zero
    ndvi_patch = (ndvi_patch - ndvi_min) / (ndvi_max - ndvi_min) if ndvi_max != ndvi_min else ndvi_patch
    temp_patch = (temp_patch - temp_min) / (temp_max - temp_min) if temp_max != temp_min else temp_patch
    evi_patch = (evi_patch - evi_min) / (evi_max - evi_min) if evi_max != evi_min else evi_patch

    return ndvi_patch, temp_patch, evi_patch

def remove_outliers(patch, lower_percentile=2.5, upper_percentile=97.5):
    """
    Removes outliers by clipping values outside of the specified percentiles
    """

    patch = np.nan_to_num(patch, nan=0)

    lower_bound = np.percentile(patch, lower_percentile)
    upper_bound = np.percentile(patch, upper_percentile)

    # Clip values to fall between lower and upper bounds
    patch = np.clip(patch, lower_bound, upper_bound)

    return patch


def generate_augmented_patches(ndvi_patch, temp_patch, evi_patch, fire_patch):
    augmented = []
    for flip in [None, 'h', 'v']:
        for angle in [0, 90, 180, 270]:
            ndvi_aug = ndvi_patch.copy()
            temp_aug = temp_patch.copy()
            evi_aug = evi_patch.copy()
            fire_aug = fire_patch.copy()

            if flip == 'h':
                ndvi_aug = np.fliplr(ndvi_aug)
                temp_aug = np.fliplr(temp_aug)
                evi_aug = np.fliplr(evi_aug)
                fire_aug = np.fliplr(fire_aug)
            elif flip == 'v':
                ndvi_aug = np.flipud(ndvi_aug)
                temp_aug = np.flipud(temp_aug)
                evi_aug = np.flipud(evi_aug)
                fire_aug = np.flipud(fire_aug)

            if angle != 0:
                ndvi_aug = rotate(ndvi_aug, angle, reshape=False)
                temp_aug = rotate(temp_aug, angle, reshape=False)
                evi_aug = rotate(evi_aug, angle, reshape=False)
                fire_aug = rotate(fire_aug, angle, reshape=False, order=0)

            input_aug = np.stack([ndvi_aug, temp_aug, evi_aug], axis=-1)  # (128,128,3)

            severity_aug = np.zeros_like(fire_patch)
            severity_aug[fire_aug == 7] = 1
            severity_aug[fire_aug == 8] = 2
            severity_aug[fire_aug == 9] = 3

            augmented.append((input_aug, severity_aug))
    return augmented


dataset_folder = "/content/drive/MyDrive/NEW_MODIS_Combined/"

tif_files = [f for f in os.listdir(dataset_folder) if f.endswith('.tif')]
patch_size = 128
threshold = 0.80

X = []
y = []

for tif_file in tif_files:
    file_path = os.path.join(dataset_folder, tif_file)

    with rasterio.open(file_path) as src:
        ndvi_mask = src.read(1)  # Band 1: NDVI
        fire_mask = src.read(2)  # Band 2: FireMask
        temp_mask = src.read(3)  # Band 3: LST
        evi_mask = src.read(4)   # Band 4: EVI

    height, width = fire_mask.shape
    num_rows = height // patch_size
    num_cols = width // patch_size

    for i in range(num_rows):
        for j in range(num_cols):
            y_start, y_end = i * patch_size, (i + 1) * patch_size
            x_start, x_end = j * patch_size, (j + 1) * patch_size

            fire_patch = fire_mask[y_start:y_end, x_start:x_end]
            ndvi_patch = ndvi_mask[y_start:y_end, x_start:x_end]
            temp_patch = temp_mask[y_start:y_end, x_start:x_end]
            evi_patch = evi_mask[y_start:y_end, x_start:x_end]

            fire_mostly_empty = np.sum((fire_patch < 4)) / fire_patch.size > threshold
            ndvi_mostly_empty = np.isnan(ndvi_patch).sum() / ndvi_patch.size > threshold
            lst_mostly_empty = np.isnan(temp_patch).sum() / temp_patch.size > threshold
            evi_mostly_empty = np.isnan(evi_patch).sum() / evi_patch.size > threshold
            fire_present = np.any(np.isin(fire_patch, [7, 8, 9]))

            if fire_mostly_empty or ndvi_mostly_empty or lst_mostly_empty or evi_mostly_empty:
                continue

            # Normalize each patch
            ndvi_patch, temp_patch, evi_patch = normalize_patch(ndvi_patch, temp_patch, evi_patch)

            # Remove outliers
            ndvi_patch = remove_outliers(ndvi_patch)
            temp_patch = remove_outliers(temp_patch)
            evi_patch = remove_outliers(evi_patch)


            if fire_present:
                augmented = generate_augmented_patches(ndvi_patch, temp_patch, evi_patch, fire_patch)
                for input_aug, label_aug in augmented:
                    X.append(input_aug)
                    y.append(label_aug)
            else:
                input_patch = np.stack([ndvi_patch, temp_patch, evi_patch], axis=-1)
                severity_patch = np.zeros_like(fire_patch)  # Default: 0 (no fire)
                severity_patch[fire_patch == 7] = 1
                severity_patch[fire_patch == 8] = 2
                severity_patch[fire_patch == 9] = 3

                X.append(input_patch)
                y.append(severity_patch)



X = np.array(X)
y = np.array(y)

print(f"Final dataset size: X = {X.shape}, y = {y.shape}")

Final dataset size: X = (1377, 128, 128, 3), y = (1377, 128, 128)


In [ ]:
y[:,:,:].max()

np.float32(3.0)

In [ ]:
# Save
np.savez_compressed(f"/content/drive/MyDrive/NEW_MODIS_Combined/new_data.npz", X=X, y=y)

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input, ZeroPadding2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import InceptionResNetV2
import matplotlib.pyplot as plt
import random
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras import backend as K

In [ ]:
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver("j")
# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

BIMO's CODE TRIAL

In [ ]:
# New Vince Data (augmented ~400 new fire patches)
data = np.load("/content/drive/MyDrive/new_data.npz")
X = data["X"]
y = data["y"]

In [ ]:
X.shape

(1377, 128, 128, 3)

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Conv2DTranspose, Concatenate, ZeroPadding2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Conv2D, BatchNormalization, Activation,
    UpSampling2D, Concatenate, ZeroPadding2D, Layer
)
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50V2

def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

class ResizeLayer(Layer):
    def call(self, input):
        input_tensor, target_tensor = input
        target_height = tf.shape(target_tensor)[1]
        target_width = tf.shape(target_tensor)[2]
        return tf.image.resize(input_tensor, [target_height, target_width])

def decoder_block(input_tensor, skip_tensor, num_filters):
    x = ResizeLayer()([input_tensor, skip_tensor])
    x = Concatenate()([x, skip_tensor])
    x = conv_block(x, num_filters)
    return x

def build_resnetv2_unet(input_shape=(128, 128, 3)):
    inputs = Input(input_shape)

    """ Pre-trained ResNetV2 Encoder """
    base_model = ResNet50V2(include_top=False, weights="imagenet", input_tensor=inputs)

    # Encoder feature maps (you can verify with model.summary())
    s1 = base_model.get_layer("conv1_conv").output       # 64x64
    s2 = base_model.get_layer("conv2_block3_out").output # 32x32
    s3 = base_model.get_layer("conv3_block4_out").output # 16x16
    s4 = base_model.get_layer("conv4_block6_out").output # 8x8
    b1 = base_model.get_layer("conv5_block3_out").output # 4x4 (bridge)

    """ Decoder """
    d1 = decoder_block(b1, s4, 512)  # 4→8
    d2 = decoder_block(d1, s3, 256)  # 8→16
    d3 = decoder_block(d2, s2, 128)  # 16→32
    d4 = decoder_block(d3, s1, 64)   # 32→64

    # Final upsampling to match input 128x128
    x = UpSampling2D((2, 2))(d4)     # 64→128
    x = Conv2D(32, 3, padding="same", activation="relu")(x)

    # outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(x)
    outputs = Conv2D(4, 1, padding="same", activation="softmax")(x)

    model = Model(inputs, outputs, name="ResNetV2_U-Net")
    return model


In [ ]:
input_shape = X.shape[1:]
model = build_resnetv2_unet(input_shape)

94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.losses import SparseCategoricalCrossentropy
import tensorflow.keras.backend as K

# def focal_loss(gamma=2., alpha=0.25):
#     def loss(y_true, y_pred):
#         epsilon = K.epsilon()
#         y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
#         pt = tf.where(K.equal(y_true, 1), y_pred, 1 - y_pred)
#         return -K.mean(alpha * K.pow(1. - pt, gamma) * K.log(pt))
#     return loss


def focal_loss(alpha=0.25, gamma=2.0):
    def loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.int32)
        y_true = tf.one_hot(y_true, depth=4)  # Convert to one-hot
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())

        # Calculate focal loss
        cross_entropy = -y_true * K.log(y_pred)
        weight = alpha * K.pow(1. - y_pred, gamma)
        focal_loss = weight * cross_entropy

        return K.mean(focal_loss)
    return loss

def dice_coefficient(y_true, y_pred, smooth=1e-6):
    # y_true: integer labels (sparse) [batch, H, W]
    # y_pred: probabilities [batch, H, W, num_classes]
    y_true = K.cast(y_true, 'int32')
    y_true_one_hot = K.one_hot(y_true, num_classes=4)  # Convert to one-hot [batch, H, W, 4]
    y_pred = K.cast(y_pred, 'float32')

    # Compute intersection and union for each class
    intersection = K.sum(y_true_one_hot * y_pred, axis=[1, 2])
    union = K.sum(y_true_one_hot, axis=[1, 2]) + K.sum(y_pred, axis=[1, 2])

    # Dice score per class, averaged over batch
    dice = K.mean((2. * intersection + smooth) / (union + smooth), axis=0)
    return K.mean(dice)  # Average over classes

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.5, verbose=1),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
]

In [ ]:
from keras.metrics import MeanIoU
class MeanIoUMetric(tf.keras.metrics.MeanIoU):
    def __init__(self, num_classes, name='mean_iou', **kwargs):
        super().__init__(num_classes=num_classes, name=name, **kwargs)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.argmax(y_pred, axis=-1)
        # No need to squeeze y_true if it's already shape (batch, H, W)
        return super().update_state(y_true, y_pred, sample_weight)
# # model.compile(
# #     optimizer='adam',
# #     loss=SparseCategoricalCrossentropy(from_logits=False),
# #     metrics=['Recall']
# # )

# model.compile(
#     optimizer='adam',
#     loss=SparseCategoricalCrossentropy(from_logits=False),
#     metrics=['accuracy']
# )


model.compile(
    optimizer='adam',
    loss=focal_loss(alpha=0.75, gamma=3.0),
    metrics=[ "accuracy", dice_coefficient, MeanIoUMetric(num_classes=4)]
)

# Train the model
model.fit(X, y, epochs=5, callbacks=callbacks)


Epoch 1/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 908s 19s/step - accuracy: 0.9999 - dice_coefficient: 0.2461 - loss: 1.0346e-04 - mean_iou: 0.2518 - learning_rate: 0.0010
Epoch 2/5


/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/callback_list.py:145: UserWarning: Learning rate reduction is conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,dice_coefficient,loss,mean_iou,learning_rate.
  callback.on_epoch_end(epoch, logs)
/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,dice_coefficient,loss,mean_iou,learning_rate
  current = self.get_monitor_value(logs)


44/44 ━━━━━━━━━━━━━━━━━━━━ 842s 19s/step - accuracy: 0.9999 - dice_coefficient: 0.2440 - loss: 7.6957e-05 - mean_iou: 0.2500 - learning_rate: 0.0010
Epoch 3/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 866s 19s/step - accuracy: 0.9999 - dice_coefficient: 0.2441 - loss: 6.9834e-05 - mean_iou: 0.2596 - learning_rate: 0.0010
Epoch 4/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 854s 19s/step - accuracy: 0.9999 - dice_coefficient: 0.2447 - loss: 8.7383e-05 - mean_iou: 0.2500 - learning_rate: 0.0010
Epoch 5/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 860s 19s/step - accuracy: 0.9999 - dice_coefficient: 0.2444 - loss: 6.2686e-05 - mean_iou: 0.2537 - learning_rate: 0.0010


In [ ]:
print(f"Final dataset size: X = {X.shape}, y = {y.shape}")

Final dataset size: X = (82, 128, 128, 3), y = (82, 128, 128)


In [ ]:
y_pred = model.predict(X_clean[:1])  # Check first sample
print("Model output:", y_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Model output: [[[[nan nan nan nan]
   [nan nan nan nan]
   [nan nan nan nan]
   ...
   [nan nan nan nan]
   [nan nan nan nan]
   [nan nan nan nan]]

  [[nan nan nan nan]
   [nan nan nan nan]
   [nan nan nan nan]
   ...
   [nan nan nan nan]
   [nan nan nan nan]
   [nan nan nan nan]]

  [[nan nan nan nan]
   [nan nan nan nan]
   [nan nan nan nan]
   ...
   [nan nan nan nan]
   [nan nan nan nan]
   [nan nan nan nan]]

  ...

  [[nan nan nan nan]
   [nan nan nan nan]
   [nan nan nan nan]
   ...
   [nan nan nan nan]
   [nan nan nan nan]
   [nan nan nan nan]]

  [[nan nan nan nan]
   [nan nan nan nan]
   [nan nan nan nan]
   ...
   [nan nan nan nan]
   [nan nan nan nan]
   [nan nan nan nan]]

  [[nan nan nan nan]
   [nan nan nan nan]
   [nan nan nan nan]
   ...
   [nan nan nan nan]
   [nan nan nan nan]
   [nan nan nan nan]]]]


In [ ]:
history = model.fit(X, y, validation_split=0.1, batch_size=16, epochs=10, callbacks=[model_checkpoint, reduce_lr_on_plateau, early_stopping])

NameError: name 'model_checkpoint' is not defined